# Training PPO for CartPole

Used [PPOPolicy from Tianshou](https://tianshou.org/en/stable/03_api/policy/modelfree/ppo.html).

In [1]:
%load_ext autoreload
%autoreload 2

# Install dependencies

In [2]:
!pip install gymnasium==0.29.1 pygame==2.3.0 pettingzoo==1.24.3 tianshou==0.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 847.8/847.8 kB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 19.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1

# Setup environment

In [15]:
import gymnasium as gym
import torch

import tianshou as ts

device = "cuda" if torch.cuda.is_available() else "cpu"

def get_env(render_mode=None):
  return gym.make("CartPole-v1", render_mode=render_mode)

env = get_env()

In [16]:
train_envs = ts.env.DummyVectorEnv([get_env for _ in range(10)])
test_envs = ts.env.DummyVectorEnv([get_env for _ in range(100)])

# Create Policy

In [17]:
state_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n

net = ts.utils.net.common.Net(state_shape=state_shape, hidden_sizes=[64, 64], device=device)

actor = ts.utils.net.discrete.Actor(preprocess_net=net, action_shape=env.action_space.n, device=device).to(device)
critic = ts.utils.net.discrete.Critic(preprocess_net=net, device=device).to(device)
actor_critic = ts.utils.net.common.ActorCritic(actor, critic)
optim = torch.optim.Adam(actor_critic.parameters(), lr=0.0003)

policy = ts.policy.PPOPolicy(
    actor=actor,
    critic=critic,
    optim=optim,
    dist_fn=torch.distributions.Categorical,
    action_space=env.action_space,
    action_scaling=False,
)

In [18]:
train_collector = ts.data.Collector(
    policy, train_envs, ts.data.VectorReplayBuffer(20000, 10), exploration_noise=True
)
test_collector = ts.data.Collector(policy, test_envs, exploration_noise=True)

# Train the agent
Using the OffPolicyTrainer.

In [20]:
result = ts.trainer.OnpolicyTrainer(
    policy=policy,
    batch_size=256,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=10,
    step_per_epoch=50000,
    repeat_per_collect=10,
    episode_per_test=10,
    step_per_collect=2000,
    stop_fn=lambda mean_reward: mean_reward >= 500,
).run()
print(f'Finished training! Use {result["duration"]}')

result

Epoch #1: 50001it [00:16, 3005.81it/s, env_step=50000, len=157, loss=57.326, loss/clip=-0.004, loss/ent=0.584, loss/vf=114.670, n/ep=14, n/st=2000, rew=157.43]                           


Epoch #1: test_reward: 143.300000 ± 83.316325, best_reward: 143.300000 ± 83.316325 in #1


Epoch #2: 50001it [00:17, 2881.67it/s, env_step=100000, len=160, loss=57.699, loss/clip=0.003, loss/ent=0.570, loss/vf=115.403, n/ep=17, n/st=2000, rew=160.88]                           


Epoch #2: test_reward: 145.500000 ± 102.473655, best_reward: 145.500000 ± 102.473655 in #2


Epoch #3: 50001it [00:16, 3100.44it/s, env_step=150000, len=169, loss=48.687, loss/clip=0.000, loss/ent=0.547, loss/vf=97.383, n/ep=11, n/st=2000, rew=169.64]                           


...

Epoch #8: 50001it [00:16, 2948.04it/s, env_step=400000, len=275, loss=48.700, loss/clip=-0.000, loss/ent=0.555, loss/vf=97.411, n/ep=10, n/st=2000, rew=275.00]                           


Epoch #8: test_reward: 117.900000 ± 60.769153, best_reward: 387.500000 ± 175.150935 in #6


Epoch #9:  52%|#####2    | 26000/50000 [00:08<00:07, 3011.79it/s, env_step=426000, len=500, n/ep=4, n/st=2000, rew=500.00]

Finished training! Use 148.60s


{'duration': '148.60s',
 'train_time/model': '76.91s',
 'test_step': 35838,
 'test_episode': 130,
 'test_time': '6.93s',
 'test_speed': '5168.02 step/s',
 'best_reward': 500.0,
 'best_result': '500.00 ± 0.00',
 'train_step': 426000,
 'train_episode': 2886,
 'train_time/collector': '64.75s',
 'train_speed': '3007.10 step/s'}

# Play

Play with the trained agent to the opponent a number of episodes and print the results

In [21]:
policy.eval()

env = get_env(render_mode=None)
env = ts.env.DummyVectorEnv([lambda: env])
collector = ts.data.Collector(policy, env, exploration_noise=True)
result = collector.collect(n_episode=100, render=None)
rews, lens = result["rews"], result["lens"]

display(rews.mean())

480.35

Plot Result

In [22]:
import plotly.figure_factory as ff

fig = ff.create_distplot([result['rews']], ['reward'])
fig.update_layout(title_text='CartPole DQN Result')
fig.show()